In [26]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly import figure_factory as figfac
from termcolor import colored

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,LSTM
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import math
from sklearn.metrics import mean_squared_error
#!pip install keras-tuner --upgrade
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

# to ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [27]:
df = pd.read_csv("gold.csv")


In [28]:
df.rename(columns = {"Date" : "date", "Open" : "open", "High" : "high", "Low" : "low",
                     "Close" : "close", "Volume" : "volume", "Currency" : "currency"}, inplace = True)

In [29]:
data = df.filter(['close'])
data.head()

,close
0,283.7
1,282.1
2,282.4
3,282.9
4,282.7


In [30]:
dataset = data.values
dataset[0:5]

array([[283.7],
       [282.1],
       [282.4],
       [282.9],
       [282.7]])

In [31]:
training_data_len = int(np.ceil( len(dataset) * .95 ))
print("Training data length: ", training_data_len)

Training data length:  5418


In [32]:
scaler = MinMaxScaler(feature_range = (0, 1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

array([[0.01494925],
       [0.01406664],
       [0.01423213],
       ...,
       [0.81067961],
       [0.80135702],
       [0.80891439]])

In [33]:
# Create the training data set 
# Create the scaled training data set
train_data = scaled_data[0:int(training_data_len), :]
# Split the data into x_train and y_train data sets
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<= 61:
        print(x_train, "\n")
        print(y_train)

[array([0.01494925, 0.01406664, 0.01423213, 0.01450794, 0.01439762,
       0.01533539, 0.01494925, 0.01572154, 0.01561121, 0.01820388,
       0.01859003, 0.01803839, 0.01825905, 0.01737643, 0.01654898,
       0.01649382, 0.0168248 , 0.01621801, 0.01632833, 0.01588703,
       0.01715578, 0.01836937, 0.03111209, 0.02642321, 0.02487864,
       0.02868491, 0.0342564 , 0.03144307, 0.03000883, 0.02620256,
       0.02669903, 0.02603707, 0.02796778, 0.02818844, 0.02526478,
       0.02443733, 0.02096205, 0.02074139, 0.02074139, 0.02024492,
       0.01825905, 0.01859003, 0.01809356, 0.02046558, 0.01864519,
       0.01991395, 0.01842454, 0.01925199, 0.01825905, 0.01820388,
       0.01676964, 0.01566637, 0.01649382, 0.01864519, 0.01748676,
       0.01588703, 0.01572154, 0.01318402, 0.01263239, 0.01064651])] 

[0.012411738746690215]
[array([0.01494925, 0.01406664, 0.01423213, 0.01450794, 0.01439762,
       0.01533539, 0.01494925, 0.01572154, 0.01561121, 0.01820388,
       0.01859003, 0.01803839, 0.

In [34]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [35]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
print("The sape of x_train set:", x_train.shape)

The sape of x_train set: (5358, 60, 1)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(128, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(64, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer = "adam", loss = "mean_squared_error")

model.fit(x_train, y_train, batch_size = 1, epochs = 1)

1899/5358 [=========>....................] - ETA: 2:08 - loss: 0.0012

In [ ]:
# Create the testing data set
# Create a new array containing scaled values from index 1543 to 2002 
test_data = scaled_data[training_data_len - 60: , :]
# Create the data sets x_test and y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])

In [ ]:
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
predictions[0:10]

In [ ]:
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
print("RMSE score of the model:", rmse)

In [ ]:
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions
# Visualize the data
plt.figure(figsize=(16,6))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['close'])
plt.plot(valid[['close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc = 'lower right')
plt.show()

In [ ]:
valid.head(n = 10)